#### Setup

In [1]:
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Interactive cell for pandas
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [7]:
import gdown
gdown.download_folder(id="1pXY7Tp6hArE6j7C0M7E-OXUXAkzevotG", quiet=True, use_cookies=False)

['/content/hackernews/models/20221028',
 '/content/hackernews/raw/hackernews-2019-2022-sessions.csv',
 '/content/hackernews/raw/hackernews-data-from-phone.json',
 '/content/hackernews/raw/hackernews-since-20221016.json',
 '/content/hackernews/raw/hackernews-stories-since-2018.csv',
 '/content/hackernews/raw/hackernews-stories-since-2022-10-14.csv',
 '/content/hackernews/raw/hackernews-urls-from-browser-deduplicated.csv',
 '/content/hackernews/tests/test_data_with_topics.csv',
 '/content/hackernews/tests/test_data.csv',
 '/content/hackernews/tests/test_set_with_topics.csv',
 '/content/hackernews/tests/test_set.csv',
 '/content/hackernews/tests/validation_set_with_topics.csv',
 '/content/hackernews/tests/validation_set.csv']

In [4]:
# !mkdir hackernews
# !mkdir hackernews/raw
# !mv hackernews-* hackernews/raw

In [8]:
# raw data
stories_2018 = f'/content/hackernews/raw/hackernews-stories-since-2018.csv'

In [11]:
df_2018 = pd.read_csv(stories_2018,
                      usecols=['title', 'url', 'id', 'timestamp_formatted'],
                      parse_dates=['timestamp_formatted'])
df_2018 = df_2018.rename(columns={"timestamp_formatted": "ts"})
df_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1726813 entries, 0 to 1726812
Data columns (total 4 columns):
 #   Column  Dtype              
---  ------  -----              
 0   title   object             
 1   url     object             
 2   id      int64              
 3   ts      datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), int64(1), object(2)
memory usage: 52.7+ MB


#### Exploration & Cleaning

**Notes for future me**

(1) Exploration are done using method chaining for fast iteration on the data
    (data are transformed and displayed without actually stored, signal options)

(2) Several approach I've just discovered might look weird, but somehow works:
    - Using `.head(n)` makes sure I'm operating on the same sample of the data
    - Chaining `apply` happens because I can't preserve values as columns
      (I wonder if this indicate the need to just create a function?
       but, what if it's an apply to avoid errors?)
    - The logic inside the `lambda` are frequently "swapped" based on whether I'm
      operating on dirty or clean data. For example, when exploring dirty data,
      I use "True if" to filter correct values and drop them with `dropna` because I
      want to see my transformation on the dirty columns. But when I finished
      cleaning, I have to "swap" the logic back to "else True" so that dirty data
      are the ones that is actually dropped (in this case I preserve the indexes)
      Variations of these patterns arise in the arithmetic comparison too

      example when cleaning one-word title:
        exploring dirty data: `True if len(sentence) == 1 else None`  # True and None
        keeping clean data  : `None if len(sentence) == 1 else True`  # is swapped

(3) Anyway, this data is MUCH. MORE DIRTY than I'd expected. Seriously didn't expect
    to spend this much time wrangling :/ really, bless community upvotes I guess
    (now that I thought about it, maybe the "dirty" patterns arises in the number
     of upvotes, kids, or empty urls? now I regret didn't get all the data hmm)

##### Duplicate Titles :)

In [12]:
df_2018.shape

(1726813, 4)

In [13]:
(
    df_2018["title"].value_counts()[:30]
)

Y Combinator: Bookmarklet                                                 1558
Books for kids (death and bullying)                                        262
Test                                                                       119
WP Paint – WordPress Image Editor Free Version                             105
Abortion Is Murder (Available on Spotify and iTunes  By, Evon Latrail)      98
ghost                                                                       97
Keybase                                                                     83
Referral URL                                                                74
GraphQL with NodeJs: From Beginner to Advanced Concepts                     67
React Native Debug Tool                                                     60
Super Affiliate System                                                      59
Stowaway – Multi-hop Proxy Tool for pentesters                              58
Check Out 'One Punch Man – Home Workout'            

In [14]:
df_2018 = (
            df_2018.drop_duplicates(subset=['title'])
                   .reset_index(drop=True)
)

In [15]:
df_2018.shape

(1564079, 4)

##### One-word Titles

In [16]:
# Identify what "one-word" title seems to be
# spoiler: it's trash
f = (
    df_2018["title"]
        .head(500)
        .str
        .split()
        .apply(lambda x: len(x) if type(x) == list else True)  # weird float error :/
        .apply(lambda x: None if x > 1 else True)
        .dropna()
        .index
)
f

Int64Index([5, 12, 70, 131, 148, 236, 285, 339, 411, 445], dtype='int64')

In [17]:
df_2018.loc[f, 'title']

5                                                 Solve
12                                        SuperbowlEVE/
70                                                  Kek
131                                              Crypto
148                     -kazakhstan-u20-vs-usa-u20-live
236                                                 Aho
285    &#20154;&#29983;&#23601;&#26159;&#36825;&#26679;
339                                            Captable
411                                                Fwef
445                                               Nabil
Name: title, dtype: object

In [18]:
preserved_indexes = (
                        df_2018["title"]
                            .str.split()
                            .apply(lambda x: len(x) if type(x) == list else True)
                            .apply(lambda x: None if x == 1 else x)
                            .dropna()
                            .index
)
preserved_indexes[:15]

Int64Index([0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16], dtype='int64')

In [19]:
df_2018 = df_2018.loc[preserved_indexes].copy().reset_index(drop=True)
df_2018.shape

(1548517, 4)

##### Job postings in Titles

In [20]:
# Identify wrong type column, e.g. job postings that are posted on story
# typically has several dashes (minimum 3) on its title
# e.g. Company — Sites or Roles — Status — Other Criteria
#
# examples
"""
'Anyroad – Https://www.anyroad.com – Full-Time – San Francisco, CA ONSITE',
'Quobyte – Santa Clara, CA and Berlin, Germany – Full-Time – Onsite',
'Wyre – Senior Full Stack Engineers – Onsite – San Francisco, CA – $100-120k',
'BCG Gamma – Senior Software Engineer – Paris or London – Full-Time – Onsite',
'Ask HN: Best way to prepare for an Onsite interview at a Startup?',
'Tabella – Ethical Hacker – Full-Time – Onsite – Prague, Czechia (EU)',
'Sama AI – Sr Software Developer (Front-End) – Onsite (Mtl) or Remote – Full Time',
'Olo – Multiple Jobs Available (see Below) – Full-Time| REMOTE or ONSITE (NYC)'
"""

f = (
        df_2018['title']
            .head(5000)
            .str.lower()
            .str.count('–')
            .apply(lambda x: None if x < 2 else True)
            .dropna()
            .index
)
df_2018.head(5000).loc[f, 'title'].values

array(['Anyroad – Https://www.anyroad.com – Full-Time – San Francisco, CA ONSITE',
       'Revved – Engineer – Cofounder – Delhi/NCR, India',
       'We Are Hiring – Earn Rs.15000/- per Month – Simple Copy Paste Jobs',
       'Twitch (Amazon) – Software Engineers – San Francisco or Remote (US/Europe) – FTE',
       'Quobyte – Santa Clara, CA and Berlin, Germany – Full-Time – Onsite',
       'Maurice – Open Beta – Serverless Load Testing Tool',
       'AWS Blog Posts – Different Authors – Same Content?',
       'Yocan Evolve Plus – Yocan Evolve Plus Review – Justchillglass $39.99',
       'SmartKarma – Singapore| Full-Time Contract – React Native Developer',
       'Qliiq.com – Save Your Bookmarks in Qliiq.com – Seek for Feedback',
       'Wyre – Senior Full Stack Engineers – Onsite – San Francisco, CA – $100-120k',
       'BCG Gamma – Senior Software Engineer – Paris or London – Full-Time – Onsite',
       'BeFit Keto Cut– {Update 2020 Reviews} – Ingredients|Prise|Does It Work',
      

In [21]:
# extract clean indexes
preserved_indexes = (
        df_2018['title']
            .str.count('–')
            .apply(lambda x: None if x > 1 else True)
            .dropna()
            .index
)
df_2018.loc[preserved_indexes, 'title'].values

array(['Ask HN: Strategies to improve after finishing a product',
       "Ask HN: What to do with 10% of UK LTD i can't work at anymore",
       'Ask HN: What can I include in an effective communication workshop?',
       ...,
       "Global Forest Watch – Discover the world's forests through data",
       'Search Google inside sheets and excel',
       'Gov trying to ban magnet balls again'], dtype=object)

In [22]:
df_2018 = df_2018.loc[preserved_indexes].copy().reset_index(drop=True)

In [23]:
df_2018.shape

(1542775, 4)

##### Links in Title

In [24]:
# Identify trash link in the title, from "http" keyword
f = (
    df_2018['title'].head(1000)
                    .str.lower()
                    .str.contains('https?:\s*')
                    .values
)
df_2018.head(1000).loc[f].title.values

array(['HTTP: //wintersupplement.com/smart-blood-sugar-book/',
       'HTTP: //Www.zzlcdz.com/',
       'HTTP: //Wintersupplement.com/Vital-Xl/',
       'HTTPS: //Keto4us.org/Trubodx-Keto/',
       'HTTPS: //Buddysupplement.com/Biorexin/',
       'HTTPS: //www.annews24.com/top-10-most-expensive-watches-in-the-world/',
       'Essence CBD Oil HTTPS://buddysupplement.com/essence-cbd-oil-au/',
       'Visit Here HTTPS://djsupplement.com/empowered-boost-testosterone/',
       'HTTPS: //Djsupplement.com/Keto-Nutrition/',
       'Buy HTTPS://www.facebook.com/Trim-Life-Keto-101398769051708'],
      dtype=object)

In [25]:
# Invert the match to get clean values
f = (
    df_2018['title'].str.lower()
                    .str.contains('https?:\s*')
                    .values
)
df_2018 = df_2018.loc[~f].copy()

In [26]:
df_2018.shape

(1541923, 4)

##### Non-stories ([Something] HN)

In [27]:
# explore dirty titles
(
    df_2018['title'].head(50)
                    .str
                    .split("HN: ")
                    .apply(lambda x: x[0].strip())
                    .value_counts()
)

Ask                                                                                37
We need help with all aspects of marketing/ Contract help needed                    1
When PMF meets market: $2.7M ARR ACHIEVED                                           1
Tell                                                                                1
Corporation Warfare-Protonmail cyberattack sponsored by states and corporations     1
The impact of Airbnb on residential real estate price rises                         1
Show                                                                                1
It's Official. GitHub is slow                                                       1
Aspiring YC founder – Can you get into YC without a technical cofounder?            1
Looking for Job/Work Contract (Web Developer Javascript/PHP)                        1
What podcast hosting provides the best analytics?                                   1
What is the state of nanotech?                        

In [28]:
# clean the titles
(
    df_2018['title'].head(20)
                    .str
                    .split("HN: ")
                    .apply(lambda x: x[-1])  # take the last one
)

0       Strategies to improve after finishing a product
1     What to do with 10% of UK LTD i can't work at ...
2     What can I include in an effective communicati...
3     We need help with all aspects of marketing/ Co...
4           Cost Effective Options for Email Marketing?
5     Should I sell equity in a past startup on the ...
6     How to get the flexibilty of containers withou...
7     Why didn't the early web support more client s...
8     Something like Khan Academy but full curriculu...
9        What's the most fun tradition at your startup?
10            When PMF meets market: $2.7M ARR ACHIEVED
11    Programming book recommendations for autistic ...
12            Enterprise Software Developers in London?
13    Have you used Adderall or any other similar drug?
14         Why landline make crazy noise while hung up?
15    Where to Learn Kubernetes, Ansible and Terraform?
16    Banned site-wide from Reddit for helping a fel...
17    Is it possible to donate small amounts to 

In [29]:
df_2018['title'] = df_2018['title'].str.split("HN: ").apply(lambda x: x[-1])

##### Last Word [Year] and [PDF]

In [30]:
dirty_last_word_examples = ['A First Lesson in Econometrics (1970) [pdf]',
                    'Monarch: Google’s Planet-Scale In-Memory Time Series Database [pdf]',
                    'High System Load with Low CPU Utilization on Linux? (2020)']

In [35]:
# examples of cleaning dirty last word
pat = r'\s\(\d+\)|\s\[pdf\]'

(
    df_2018
        .loc[
            df_2018['title'].isin(dirty_last_word_examples),
            'title'
        ]
        .str
        .replace(pat, '', regex=True)
        .values
)

array([], dtype=object)

In [32]:
df_2018['title'] = df_2018['title'].str.replace(pat, '', regex=True)

##### Non-English language

In [ ]:
"""
Don't think I'll be using this for now because it might not work properly
with short texts according to the docs, which can be seen below, and my
tokenizer is probably good enough at this point (could recognize HN stuffs)
"""
f = (
    df_2018['title'].head(100)
        .apply(detect)
        .apply(lambda x: None if x == 'en' else 1)
        .dropna()
        .index
)
f

In [ ]:
df_2018.loc[f, 'title']

In [ ]:
(
    df_2018
        .loc[f, 'title']
        .apply(detect_langs)
)